In [ ]:
%%sql -r dataframe_1
-- JsonRawデータ取り込みsnowpipe (FLATTENは使えない！
create or replace pipe "TEST"."TEST_SCHEMA"."PIPE_RAW_JSON_VARIANT"
  auto_ingest = true
as
copy into "TEST"."TEST_SCHEMA"."RAW_JSON_VARIANT" (PAYLOAD, LOAD_TS)
from (
  select
    $1::variant                      as PAYLOAD,
    METADATA$START_SCAN_TIME::timestamp_ntz as LOAD_TS
  from @"TEST"."TEST_SCHEMA"."INTERNAL_STAGE"/json/
)
file_format = (format_name = "TEST"."TEST_SCHEMA"."FF_JSON_STRIP")
pattern = '.*\\.json$'
on_error = 'continue';


In [ ]:
%%sql -r dataframe_2
-- Snowpipe実行状況
select system$pipe_status('TEST.TEST_SCHEMA.PIPE_RAW_JSON_VARIANT');


In [ ]:
%%sql -r dataframe_4
-- 取り込み履歴
select
  file_name,
  stage_location,
  last_load_time,
  status,
  row_count,
  error_count,
  first_error_message
from table("TEST".information_schema.copy_history(
  table_name => 'TEST_SCHEMA.RAW_JSON_VARIANT',
  start_time => dateadd('day', -30, current_timestamp())
))
order by last_load_time desc;


In [ ]:
%%sql -r dataframe_5
-- jsonをflattennにするフォーマット
create or replace file format "TEST"."TEST_SCHEMA"."FF_JSON_STRIP"
  type = json
  strip_outer_array = true
  multi_line = true;  -- 整形JSONで複数行にまたがる場合に保険


In [ ]:
%%sql -r dataframe_6
-- copy intoコマンドで取り込みする　すでに取り込み済みのは再取り込みされない
copy into "TEST"."TEST_SCHEMA"."RAW_JSON_VARIANT"(payload)
from @"TEST"."TEST_SCHEMA"."INTERNAL_STAGE"/json/
file_format = (format_name = "TEST"."TEST_SCHEMA"."FF_JSON_STRIP")
pattern = '.*\\.json$';
